In [52]:
# Imports
from datetime import datetime

import pykep as pk
import numpy as np
import pandas as pd
from pykep.orbit_plots import plot_planet, plot_lambert
from pykep import AU, DAY2SEC

# Plotting imports
from poliastro.bodies import Earth, Mars, Sun
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from poliastro.bodies import Sun
from poliastro.ephem import Ephem
from poliastro.plotting import OrbitPlotter3D
from poliastro.util import time_range
from poliastro.maneuver import Maneuver
from astropy.time import Time
from poliastro.twobody.orbit import Orbit

# Function to generate random data and calculate delta V
def generate_data(num_samples):
    data = []
    while len(data) < num_samples:
        # Generate random starting position (x, y, z) on the surface of a sphere with Earth's radius
        theta = np.random.uniform(0, 2 * np.pi)
        phi = np.random.uniform(0, np.pi)
        r = 6.371e6  # Earth's radius in meters
        start_pos = np.array([r * np.sin(phi) * np.cos(theta),
                              r * np.sin(phi) * np.sin(theta),
                              r * np.cos(phi)])

        # Generate random relative distance (dx, dy, dz) within a certain range
        rel_dist = np.random.uniform(-1e6, 1e6, size=3)

        # Generate random time of flight within a certain range
        tof = np.random.uniform(0, 1000)  # Time of flight in days

        # Calculate required initial delta V using Lambert problem function
        delta_v = calculate_delta_v(start_pos, rel_dist, tof)

        # Check if delta V is NaN
        if np.isnan(delta_v):
            continue  # Skip this data point

        # Append the data along with delta V to the list
        data.append({"start_pos_x": start_pos[0],
                     "start_pos_y": start_pos[1],
                     "start_pos_z": start_pos[2],
                     "rel_dist_x": rel_dist[0],
                     "rel_dist_y": rel_dist[1],
                     "rel_dist_z": rel_dist[2],
                     "tof": tof,
                     "delta_v": delta_v})
    return data

# Function to calculate delta V using Lambert problem
def calculate_delta_v(start_pos, rel_dist, tof):
    # Convert time of flight from days to seconds
    dt = tof * DAY2SEC

    # Solve Lambert problem
    l = pk.lambert_problem(r1=start_pos, r2=start_pos+rel_dist, tof=dt, mu=pk.MU_EARTH, max_revs=2)

    # Calculate delta V
    delta_v = np.linalg.norm(l.get_v1()[0])

    return delta_v

# Generate data
num_samples = 1000
data = generate_data(num_samples)

# Convert data to DataFrame
df = pd.DataFrame(data)

# Define the filename
filename = "dataset.csv"

# Save DataFrame to CSV
df.to_csv(filename, index=False)
print("Data has been written to", filename)

Data has been written to dataset.csv


In [53]:
# Read the CSV file into a DataFrame
df = pd.read_csv('dataset.csv')

# Display the DataFrame
print(df)


      start_pos_x   start_pos_y   start_pos_z     rel_dist_x     rel_dist_y  \
0   -1.929547e+06  3.361045e+06 -5.056665e+06 -333083.393031  858740.693456   
1   -3.282117e+05 -1.646415e+06 -6.145831e+06 -217657.676487 -762831.675373   
2   -8.725720e+05  5.743693e+05 -6.284772e+06   13924.158270   45815.772255   
3    1.345952e+06  5.791519e+06  2.288309e+06 -554200.988778  588788.391549   
4    5.586848e+06  6.573878e+05 -2.990754e+06 -941003.637668 -331248.752311   
..            ...           ...           ...            ...            ...   
995 -6.877723e+05  2.979504e+06  5.589201e+06 -261988.919329  501165.898882   
996  2.728927e+06  4.436362e+06 -3.668963e+06  437272.786842  631818.018817   
997 -1.543012e+05 -4.513971e+06  4.493317e+06  216122.402705  596785.950861   
998 -3.794776e+06  3.815520e+06 -3.410444e+06 -784897.581857 -589094.090903   
999 -1.450602e+06 -8.255566e+05  6.148484e+06  927218.043298 -969651.016890   

        rel_dist_z         tof       delta_v  
0   

In [ ]:
# WORK IN PROGRESS
from astropy import units as u
from poliastro.constants import GM_sun
from poliastro.frames import Planes


def visualize_transfer(start_pos, start_vel, final_pos, final_vel, date_launch, date_arrival):
    r = start_pos * u.km
    v = start_vel * u.km / u.s
    r2 = final_pos * u.km
    v2 = final_vel * u.km / u.s

    # Create orbits from initial and final positions
    ss_start = Orbit.from_vectors(Earth, r, v, date_launch)
    ss_final = Orbit.from_vectors(Earth, r2, v2, date_arrival)

    start = Ephem.from_orbit(ss_start, date_launch)
    finish = Ephem.from_orbit(ss_final, date_arrival)

    # Solve for the transfer maneuver
    man_lambert = Maneuver.lambert(ss_start, ss_final)

    # Apply maneuver to get transfer and final orbits
    ss_trans, ss_target = ss_start.apply_maneuver(man_lambert, intermediate=True)

    print("Delta-v for the transfer:", man_lambert.get_total_cost())
    print("Transfer orbit:")
    print(man_lambert.get_total_time())

    # Plot the transfer orbit
    plotter = OrbitPlotter3D()
    plotter.set_attractor(Earth)

    plotter.plot_ephem(start, date_launch, label="Start position")
    plotter.plot_ephem(finish, date_arrival, label="Final position")
    fig = plotter.plot_maneuver(
        ss_start,
        man_lambert,
        color="black",
        label="Transfer orbit",
    )

    # Set view parameters
    plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

    # Save the plot as HTML
    fig.write_html("plots/transfer_orbit.html")

# Generate random velocity within a certain range
min_vel = -1000  # Minimum velocity in m/s
max_vel = 1000   # Maximum velocity in m/s
start_vel = np.random.uniform(min_vel, max_vel, size=3)
end_vel = np.random.uniform(min_vel, max_vel, size=3)

# Visualize one transfer
sample_data = data[100]  # Choose the first data point
start_pos = np.array([sample_data['start_pos_x'], sample_data['start_pos_y'], sample_data['start_pos_z']])
rel_dist = np.array([sample_data['rel_dist_x'], sample_data['rel_dist_y'], sample_data['rel_dist_z']])

# Example usage
date_launch = Time("2024-04-23", scale="utc").tdb
date_arrival = Time("2024-08-23", scale="utc").tdb

visualize_transfer(start_pos, start_vel, start_pos+rel_dist, end_vel, date_launch, date_arrival)